# 관리자 결제 승인 시나리오 (Admin Payment Approval Scenario)

1. **결제 요청**: 사용자가 특정 금액과 목적에 대해 결제 승인을 요청
2. **워크플로우 일시정지**: `interrupt`를 통해 관리자의 응답을 기다림
3. **관리자 응답**: 승인 또는 거부 결정
4. **워크플로우 재개**: 관리자의 응답에 따라 프로세스 계속 진행

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.chat_models import init_chat_model

# 모델 초기화
model = init_chat_model("openai:gpt-5-mini")
# model = init_chat_model("google_genai:gemini-2.5-flash")

In [13]:
from uuid import uuid4
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.types import Command
from langchain.tools import tool
from langchain.messages import HumanMessage, SystemMessage

# 결제 승인 요청 도구 정의
@tool
def request_payment_approval(amount: int, purpose: str, requester: str) -> str:
    """
    관리자 결제 승인 요청 도구.
    HumanInTheLoopMiddleware에 의해 interrupt가 발생함.
    """
    return (
        f"결제 승인 요청 발생!\n"
        f"- 요청자: {requester}\n"
        f"- 금액: {amount:,}원\n"
        f"- 목적: {purpose}\n"
        f"관리자 결정을 대기 중..."
    )

# Human-in-the-Loop 미들웨어 정의
hitl = HumanInTheLoopMiddleware(
    interrupt_on={
        "request_payment_approval": {
            "allowed_decisions": ["approve", "reject", "edit"]
        }
    }
)

agent = create_agent(
    model,
    tools=[request_payment_approval],
    checkpointer=InMemorySaver(),
    middleware=[hitl],
)

In [14]:
requester = "홍길동"
amount = 850_000
purpose = "세미나 출장비"

# thread_id로 세션 유지
thread_id = f"payment-{uuid4()}"
config = {"configurable": {"thread_id": thread_id}}

# 결제 승인 요청 → interrupt 발생
response = agent.invoke(
    {
        "messages": [
            SystemMessage(content=(
                "당신은 결제 승인 Human-in-the-loop 에이전트입니다. "
                "인터럽트가 승인(approve)으로 해소되면, 대기 상태를 반복 설명하지 말고 "
                "즉시 '승인 완료/처리 완료' 로만 간단히 출력하세요."
            )),
            HumanMessage(
                content=(
                    f"관리자님, {requester}의 결제 요청입니다.\n"
                    f"- 금액: {amount:,}원\n- 목적: {purpose}\n"
                    "승인 여부를 판단해 주세요."
                )
            )
        ]
    },
    config=config,
)

print("[1단계] 결제 요청 → 워크플로우 일시정지\n", response)

[1단계] 결제 요청 → 워크플로우 일시정지
 {'messages': [SystemMessage(content="당신은 결제 승인 Human-in-the-loop 에이전트입니다. 인터럽트가 승인(approve)으로 해소되면, 대기 상태를 반복 설명하지 말고 즉시 '승인 완료/처리 완료' 로만 간단히 출력하세요.", additional_kwargs={}, response_metadata={}, id='1bf0a768-7d11-4fe7-9ab7-effa08917f34'), HumanMessage(content='관리자님, 홍길동의 결제 요청입니다.\n- 금액: 850,000원\n- 목적: 세미나 출장비\n승인 여부를 판단해 주세요.', additional_kwargs={}, response_metadata={}, id='49f59042-5cd0-4794-8657-0a4bbb755e74'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 361, 'prompt_tokens': 249, 'total_tokens': 610, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 320, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CVXg92FD0degSNOYmcDAWla1e4ir0', 'service_tier': 'default', 'finish_reaso

In [16]:
response["__interrupt__"]

[Interrupt(value={'action_requests': [{'name': 'request_payment_approval', 'args': {'amount': 850000, 'purpose': '세미나 출장비', 'requester': '홍길동'}, 'description': "Tool execution requires approval\n\nTool: request_payment_approval\nArgs: {'amount': 850000, 'purpose': '세미나 출장비', 'requester': '홍길동'}"}], 'review_configs': [{'action_name': 'request_payment_approval', 'allowed_decisions': ['approve', 'reject', 'edit']}]}, id='dbff0bb2574203a6217d6244347294a5')]

In [17]:
# Command로 approve 결정 전달
admin_command = Command(resume={
    "decisions": [
        {
            "tool": "request_payment_approval",   # 인터럽트가 걸린 툴 이름과 정확히 일치해야 함
            "type": "approve",           # approve / edit / reject
        }
    ]
})

resumed = agent.invoke(
    admin_command,
    config
)

print("\n[2단계] 워크플로우 재개 결과 (승인)\n", resumed)


[2단계] 워크플로우 재개 결과 (승인)
 {'messages': [SystemMessage(content="당신은 결제 승인 Human-in-the-loop 에이전트입니다. 인터럽트가 승인(approve)으로 해소되면, 대기 상태를 반복 설명하지 말고 즉시 '승인 완료/처리 완료' 로만 간단히 출력하세요.", additional_kwargs={}, response_metadata={}, id='1bf0a768-7d11-4fe7-9ab7-effa08917f34'), HumanMessage(content='관리자님, 홍길동의 결제 요청입니다.\n- 금액: 850,000원\n- 목적: 세미나 출장비\n승인 여부를 판단해 주세요.', additional_kwargs={}, response_metadata={}, id='49f59042-5cd0-4794-8657-0a4bbb755e74'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 361, 'prompt_tokens': 249, 'total_tokens': 610, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 320, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CVXg92FD0degSNOYmcDAWla1e4ir0', 'service_tier': 'default', 'finish_reason

In [18]:
resumed['messages'][-1].pretty_print()

================================== Ai Message ==================================

승인 완료
